## Setting Up AWS Environment
Create S3 Bucket

In [ ]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'customer-segmentation-bucket'
region = 'us-west-2'

s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
print(f'Bucket {bucket_name} created.')


## Data Preparation
Load and Preprocess Data

In [ ]:
import pandas as pd

# Load data
data = pd.read_csv('path/to/your/customer_data.csv')

# Preprocess data (example)
# Assume data needs normalization and handling missing values
data = data.dropna()
data = (data - data.mean()) / data.std()

# Save the preprocessed data to S3
data.to_csv('preprocessed_customer_data.csv', index=False)
s3.upload_file('preprocessed_customer_data.csv', bucket_name, 'preprocessed_customer_data.csv')
print('Data uploaded to S3.')


## Model Training
Train K-means Model with SageMaker

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
container = get_image_uri(region, 'kmeans')

# Set up the estimator
kmeans = sagemaker.estimator.Estimator(container,
                                       role,
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=f's3://{bucket_name}/output')

# Set hyperparameters
kmeans.set_hyperparameters(k=5,  # number of clusters
                           feature_dim=10,  # number of features
                           mini_batch_size=100)

# Fit the model
train_data = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket_name}/preprocessed_customer_data.csv', content_type='csv')
kmeans.fit({'train': train_data})


## Model Deployment
Deploy the Trained Model

In [ ]:
kmeans_predictor = kmeans.deploy(initial_instance_count=1, instance_type='ml.m3.xlarge')


## Prediction and Evaluation
Make Predictions

In [ ]:
import numpy as np

# Sample data for prediction
sample_data = np.array([[0.5, 0.3, ...], [0.2, 0.8, ...]])  # replace with actual data

# Predict
predictions = kmeans_predictor.predict(sample_data)
print(predictions)


## Automation and Monitoring
Set Up AWS Lambda for Automation

In [ ]:
import json
import boto3

lambda_client = boto3.client('lambda')

lambda_function_code = '''
import json
import boto3

def lambda_handler(event, context):
    sagemaker = boto3.client('sagemaker-runtime')
    response = sagemaker.invoke_endpoint(
        EndpointName='your-endpoint-name',
        Body=json.dumps(event['data']),
        ContentType='application/json'
    )
    result = json.loads(response['Body'].read().decode())
    return {
        'statusCode': 200,
        'body': json.dumps(result)
    }
'''

response = lambda_client.create_function(
    FunctionName='CustomerSegmentationLambda',
    Runtime='python3.8',
    Role='arn:aws:iam::your-account-id:role/your-lambda-role',
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': lambda_function_code}
)

print(response)
